# Task 4.6 (steps 3 - 7)

#### Importing libraries and dataframes

In [3]:
# Importing libraries
import pandas as pd
import numpy as np
import os

#### Step No. 3: In a new notebook, import the orders_products_combined dataframe from the pickle file you just saved.

In [5]:
# Importing 'orders_products_combined' from the 'Prepared data' folder
path = r'C:\Users\andd0\Documents\InstaCart Basket Analysis'

In [6]:
orders_products_combined = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))

In [7]:
df_prods_prepared = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'prods__wrangled_Ex4_5.csv'), index_col = False)

#### Step No. 4: Check the shape of the imported dataframe (it should be the same as the one you exported—always check!).

In [9]:
orders_products_combined.shape

(32434489, 11)

In [10]:
# Same number of rows: 32,434,489

#### Step No. 5: Determine a suitable way to combine the orders_products_combined dataframe with your products data set.

In [12]:
# Make sure you’re using your wrangled, cleaned, and deduped products data set stored in your “Prepared Data” folder from the previous Exercise’s task.
orders_products_combined.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,Yes,196,1,0,both
1,2539329,1,1,2,8,NaN,Yes,14084,2,0,both
2,2539329,1,1,2,8,NaN,Yes,12427,3,0,both
3,2539329,1,1,2,8,NaN,Yes,26088,4,0,both
4,2539329,1,1,2,8,NaN,Yes,26405,5,0,both


In [13]:
# Dropping some columns I won't need
df_prods_prep = df_prods_prepared.drop(columns = ['Unnamed: 0'])

In [14]:
df_prods_prep.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [15]:
df_merged_large = df_prods_prep.merge(orders_products_combined, on = 'product_id', how = 'inner', indicator = 'merge_status')

In [16]:
# I was getting this error "ValueError: Cannot use name of an existing column for indicator column"
# Cause the '_merge' column already exists
# Solution: create a custom name: 'merge_status'

#### Step No. 6: Confirm the results of the merge using the merge flag.

In [18]:
df_merged_large['merge_status'].value_counts()

merge_status
both          32434212
left_only            0
right_only           0
Name: count, dtype: int64

#### Testing with outer

In [20]:
df_merged_large_outer = df_prods_prep.merge(orders_products_combined, on = 'product_id', how = 'outer', indicator = 'merge_status')

In [21]:
df_merged_large_outer.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,add_to_cart_order,reordered,_merge,merge_status
0,1,Chocolate Sandwich Cookies,61.0,19.0,5.8,3139998.0,138.0,28.0,6.0,11.0,3.0,No,5.0,0.0,both,both
1,1,Chocolate Sandwich Cookies,61.0,19.0,5.8,1977647.0,138.0,30.0,6.0,17.0,20.0,No,1.0,1.0,both,both
2,1,Chocolate Sandwich Cookies,61.0,19.0,5.8,389851.0,709.0,2.0,0.0,21.0,6.0,No,20.0,0.0,both,both
3,1,Chocolate Sandwich Cookies,61.0,19.0,5.8,652770.0,764.0,1.0,3.0,13.0,NaN,Yes,10.0,0.0,both,both
4,1,Chocolate Sandwich Cookies,61.0,19.0,5.8,1813452.0,764.0,3.0,4.0,17.0,9.0,No,11.0,1.0,both,both


In [22]:
df_merged_large_outer['merge_status'].value_counts()

merge_status
both          32434212
right_only        2029
left_only           11
Name: count, dtype: int64

#### Differences between 'outer' join and 'inner' join

In [44]:
# The product catalog (df_prods_prep) has one row per product, so product IDs are unique.
# The order data (orders_products_combined) can have the same product ID many times, because customers can buy the same product in different orders.

#### What happened after the merge?

In [48]:
# When merging these dataframes, the resulting dataframe will include rows of df_prods_prep duplicated multiple times 
# (once for each matching product_id column from orders_products_combined).

#### What do right_only = 2029 and left_only = 11 mean?

In [51]:
# This means there are 2029 rows in orders_products_combined that did not match any product_id in df_prods_prep
# AND there are 11 products in df_prods_prep that never appear in any order in orders_products_combined.

#### What could be the cause of this?

In [55]:
# Possible reasons:

# These are new products that were added to the product catalog (df_prods_prep) but haven’t been purchased yet.

# These are discontinued products that were removed from the order system (orders_products_combined) but still exist in the catalog.

# Some product IDs may have changed / been updated, breaking the link between the two datasets.

#### Step No. 7: Export the newly created dataframe as ords_prods_merge in a suitable format (taking into consideration the size).

In [24]:
df_merged_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge.pkl'))